# EBA Data Ingestion

We need to collect data from two main sources for this project:
 - First is loading the EBA data into the SQL DB
 - Second is getting the real weather data from NOAA based off Weather stations.   
   This is done via FTP using `code/utils/get_weather_data.py` for the relevant time periods.
 - (Third would be accessing NOAA's forecast DB.)

 In all cases we will be loading the data into a Postgres Database for easier querying later.  


# Library Sketch and Table Sketch 

- 560 MB of weather station data
- 2.8GB of Energy data
- 5.0GB of forecast data  (could try to only extract station data)

Energy data x 100 ISOs
- Demand
- Demand Forecast
- Net Generation
    (by source)
- Transfers

Weather x 600 stations
- Temp
- Cloud cover
- Precipitation

Forecast
- Temp (gridded 24 hour forecast) of CONUS.  Probably don't want in DB.
- include file ref.
- Try to find nearest forecast pixel for all airports.

Given we want to think about a whole system forecast, we can live with having a few big tables separated by variable.
Use UTC time variables to allow a common index and forecast.

Demand Table
    id, 
    datetime
    iso1,
    iso2,
    ...
    index on datetime

Forecast Table
    ""

Net Generation (*)
    " " 
(same for sub-sources)

Transfers  (*)
   id, 
   datetime
   iso1,
   iso2,
   amount
   index on datetime, 

AirMeta
   id
   station_name
   lat
   long
   region
   city
   state
   
Temperature
   id, 
   datetime
   st1,
   st2,
   ...
   
   

In [2]:
import os
import sys
import pandas as pd


In [3]:
pwd

'/tf/notebooks'

# Bulk EBA data import

The EBA data can be downloaded from `https://www.eia.gov/opendata/bulk/EBA.zip`.
As of Mar 6, 2023 it's around 2.8 GB, with around 2800 child series, stored in one JSONLines files.

That's downloaded to data/EBA/20230302.  
For initial quick exploration we you can grep out 'California' and 'Portland' series to 


- grepped out all Portland files and California files for a smaller subset of data to play with while cleaning
up the ETL work
  `grep -r "Portland" EBA.txt > EBA_PDX.txt`
  `grep -r "California" EBA.txt > EBA_CA.txt`
  

In [4]:
import json
import jsonlines
import re
from tqdm import tqdm

def read_eba_txt(fn:str, N:int=None, name_lookup:str=None):
    """Read in all JSON from Lines file.

    Args:
    N - maximum number of lines to read in
    name_lookup - optional string to search for.  
    Return:
    List of dicts
    """
    count = 0
    data = []

    #name_reg = re.compile(f'{name_lookup}') if name_lookup else None
    with jsonlines.open(fn, 'r') as fh:
        for obj in tqdm(fh):
            #print(obj.get('series_id'), obj.get('name'))

            if name_lookup:
                if name_lookup.lower() in obj.get('name').lower():
                    print(f"HIT! {obj['name']}")
                    data.append(obj)
                    
            else:
                data.append(obj)
            if N and len(data) >= N:
                break
    return data


- This eats a LOT of ram on it's own for all files.  
- Probably best to ETL one at a time.  Even in dict form it's eating around 20GB of RAM.

In [4]:
eba_path = '/tf/data/EBA/EBA20230302'
fname = f'{eba_path}/EBA_PDX.txt'

In [5]:
!cd /tf/data/EBA/EBA20230302 && grep Portland EBA.txt > EBA_PDX.txt

In [6]:
all_data = read_eba_txt(fname)

32it [00:02, 13.67it/s]


In [14]:
for dat in all_data:
    if 'series_id' in dat.keys():
        print(dat['series_id'], dat['name'])
        print(len(dat['data']), dat['data'][0:2], dat['data'][-1])
        print()
    else:
        print(dat['category_id'], dat['name'], dat['childseries'])

EBA.PGE-ALL.D.H Demand for Portland General Electric Company (PGE), hourly - UTC time
66520 [['20230302T22Z', 2957], ['20230302T21Z', 3050]] ['20150722T08Z', 1936]

EBA.PGE-ALL.D.HL Demand for Portland General Electric Company (PGE), hourly - local time
66520 [['20230302T14-08', 2957], ['20230302T13-08', 3050]] ['20150722T01-07', 1936]

EBA.PGE-PACW.ID.H Actual Net Interchange for Portland General Electric Company (PGE) to PacifiCorp West (PACW), hourly - UTC time
66250 [['20230301T08Z', 84], ['20230301T07Z', 102]] ['20150721T08Z', -92]

EBA.PACW-PGE.ID.H Actual Net Interchange for PacifiCorp West (PACW) to Portland General Electric Company (PGE), hourly - UTC time
66959 [['20230301T08Z', -84], ['20230301T07Z', -102]] ['20150701T08Z', 101]

EBA.PGE-BPAT.ID.H Actual Net Interchange for Portland General Electric Company (PGE) to Bonneville Power Administration (BPAT), hourly - UTC time
66265 [['20230301T08Z', -1638], ['20230301T07Z', -1738]] ['20150721T08Z', -1268]

EBA.PGE-BPAT.ID.HL Ac

- note that the transfers are not fully aligned for the most recent data?  I suspect some sort of reconciliation procedure
clears that up?  Would need to look into that.  Useful for considering trades.

So we have 4 big categories of data in this thing.  All series are provided with local time and global time variations.

- Demand
- Demand Forecast
- Net Generation
- Net Generation (by source) - Much less data
- Total Interchange
- Interchange with other ISOs

- Around 8 years of data for demand/net generation.
- Around 5 years for generation by source data.
- Hourly resolution 
- Around 100 ISOs  (2850 series, 30 series per ISO, but variable interchanges).
- 60k data points per series at hourly resolution.

## Proposed SQL Table Structure - EBA

- Our initial project is focused on the demand forecasting piece.  Let's just focus on the bulk attributes for now, and return later if need be for
 breakdowns by generation type

### Options:
1) 1 table per series (hard to look up) - Reject.

2) 1 table per type (100 ISOs as columns).
    - Demand (Time, PDX, BPA, CAISO, ...)
    - Forecast (Time, PDX, BPA, CAISO, ...)
    - Net Generation (Time, PDX, BPA, CAISO,...)
    - Interchange(Time, P1, P2, Amount)

3) 1 major table per ISO (around 30 sub-series)
   -  PGE (Time, Demand, Forecast, Net Generation, COL, HYD, ..., PGE-BPA, PGE-PACW)
   -  BPA (Time, Demand, Forecast, Net Generation, COL, HYD, ..., BPA-PGE, PGE-PACW)

Leaning toward approach 3.  Better encapsulates system process.  Allows local time and UTC time
Also leaning towards only including UTC time variations.

- Need all series names (types of data)
- Need all ISOs and transferes.

In [5]:
from us_elec.SQL.sqldriver import EBAMeta, ISDMeta, SQLDriver


In [6]:
ebm = EBAMeta()
isdm = ISDMeta()
sqldr = SQLDriver()

## Getting Metadata

(Increasingly getting feeling that Mongo is the way to really handle this data)

## EBA

Want:
- list of ISOs, names

## Airports

I think the `merge_air_df` is probably already close to what we want: mapping from id to name/region.


In [5]:
# extract all the meta series 
#!cd /tf/data/EBA/EBA20230302 && grep childseries EBA.txt > metaseries.txt

In [6]:
#!head /tf/data/EBA/EBA20230302/metaseries.txt

In [18]:

# grep -r category_id EBA.txt > metaseries.txt
fn = '/tf/data/EBA/EBA20230302/metaseries.txt'
meta_df = pd.read_json(fn, lines=True)

In [9]:
ebm.extract_meta_data()
#ebm.save_iso_dict_json()

In [10]:
ebm.load_iso_dict_json()

{'US48': 'United States Lower 48',
 'CAL': 'California',
 'CAR': 'Carolinas',
 'CENT': 'Central',
 'FLA': 'Florida',
 'MIDA': 'Mid Atlantic',
 'MIDW': 'Midwest',
 'NW': 'Northwest',
 'SE': 'Southeast',
 'SW': 'Southwest',
 'ERCO': 'Electric Reliability Council of Texas Inc',
 'ISNE': 'ISO New England',
 'NYIS': 'New York Independent System Operator',
 'TVA': 'Tennessee Valley Authority',
 'NE': 'New England',
 'NY': 'New York',
 'TEN': 'Tennessee',
 'TEX': 'Texas',
 'AZPS': 'Arizona Public Service Company',
 'AECI': 'Associated Electric Cooperative Inc',
 'AVA': 'Avista Corporation',
 'BANC': 'Balancing Authority of Northern California',
 'BPAT': 'Bonneville Power Administration',
 'CISO': 'California Independent System Operator',
 'HST': 'City of Homestead',
 'TPWR': 'City of Tacoma Department of Public Utilities Light Division',
 'TAL': 'City of Tallahassee',
 'DUK': 'Duke Energy Carolinas',
 'FPC': 'Duke Energy Florida Inc',
 'CPLE': 'Duke Energy Progress East',
 'CPLW': 'Duke Energ

### Saving Airport Metadata

From `airport_play.ipynb` which downloaded all that data we have the merge_df which merged city and location information
with callsign info.


In [39]:
df = isdm.get_air_meta_df()

In [11]:
#isdm.save_callsigns()

In [40]:
#isdm.load_callsigns()

['PAAQ',
 'PABA',
 'PABE',
 'PABI',
 'PABL',
 'PABR',
 'PABT',
 'PABV',
 'PACD',
 'PACM',
 'PACV',
 'PACZ',
 'PADE',
 'PADK',
 'PADL',
 'PADM',
 'PADQ',
 'PADU',
 'PAED',
 'PAEH',
 'PAEI',
 'PAEM',
 'PAEN',
 'PAFA',
 'PAFB',
 'PAFE',
 'PAFM',
 'PAGA',
 'PAGK',
 'PAGM',
 'PAGS',
 'PAHC',
 'PAHL',
 'PAHN',
 'PAHO',
 'PAHP',
 'PAII',
 'PAIK',
 'PAIL',
 'PAIM',
 'PAJN',
 'PAKN',
 'PAKP',
 'PAKT',
 'PAKW',
 'PALU',
 'PAMC',
 'PAMR',
 'PAMY',
 'PANC',
 'PANI',
 'PANN',
 'PANT',
 'PANV',
 'PAOM',
 'PAOR',
 'PAOT',
 'PAPB',
 'PAPC',
 'PAPG',
 'PAPH',
 'PAPM',
 'PAQT',
 'PARC',
 'PARY',
 'PASA',
 'PASC',
 'PASD',
 'PASH',
 'PASI',
 'PASK',
 'PASM',
 'PASN',
 'PASV',
 'PASX',
 'PASY',
 'PATG',
 'PATK',
 'PATL',
 'PATQ',
 'PAVD',
 'PAVL',
 'PAWD',
 'PAWG',
 'PAWI',
 'PAWN',
 'PAWS',
 'PAYA',
 'PFYU',
 'PPIZ',
 'K79J',
 'KANB',
 'KBFM',
 'KBHM',
 'KDHN',
 'KEET',
 'KHSV',
 'KHUA',
 'KMGM',
 'KMOB',
 'KMSL',
 'KMXF',
 'KOZR',
 'KTCL',
 'KTOI',
 'KBPK',
 'KBYH',
 'KELD',
 'KFSM',
 'KFYV',
 'KHOT',
 

## Creating Tables

Now to create the actual tables.  

In [6]:
ebm.create_eba_tables()

CREATE TABLE IF NOT EXISTS demand  ( ts timestamp, US48 float, CAL float, CAR float, CENT float, FLA float, MIDA float, MIDW float, NW float, SE float, SW float, ERCO float, ISNE float, NYIS float, TVA float, NE float, NY float, TEN float, TEX float, AZPS float, AECI float, AVA float, BANC float, BPAT float, CISO float, HST float, TPWR float, TAL float, DUK float, FPC float, CPLE float, CPLW float, EPE float, FMPP float, FPL float, GVL float, IPCO float, IID float, JEA float, LDWP float, LGEE float, MISO float, NEVP float, NWMT float, OVEC float, PJM float, DOPD float, PACE float, PACW float, PGE float, AEC float, PSCO float, PNM float, CHPD float, GCPD float, PSEI float, SRP float, SCL float, SEC float, SCEG float, SC float, SOCO float, SWPP float, SPA float, TEC float, TEPC float, TIDC float, NSB float, WALC float, WACM float, WAUW float, YAD float, DEAA float, AVRN float, EEI float, GRMA float, GRID float, GRIF float, GWA float, WWA float, HGMA float, SEPA float, GLHB float );
CREAT

In [7]:
isdm.create_isd_tables()

CREATE TABLE IF NOT EXISTS temperature  ( ts timestamp, PAAQ float, PABA float, PABE float, PABI float, PABL float, PABR float, PABT float, PABV float, PACD float, PACM float, PACV float, PACZ float, PADE float, PADK float, PADL float, PADM float, PADQ float, PADU float, PAED float, PAEH float, PAEI float, PAEM float, PAEN float, PAFA float, PAFB float, PAFE float, PAFM float, PAGA float, PAGK float, PAGM float, PAGS float, PAHC float, PAHL float, PAHN float, PAHO float, PAHP float, PAII float, PAIK float, PAIL float, PAIM float, PAJN float, PAKN float, PAKP float, PAKT float, PAKW float, PALU float, PAMC float, PAMR float, PAMY float, PANC float, PANI float, PANN float, PANT float, PANV float, PAOM float, PAOR float, PAOT float, PAPB float, PAPC float, PAPG float, PAPH float, PAPM float, PAQT float, PARC float, PARY float, PASA float, PASC float, PASD float, PASH float, PASI float, PASK float, PASM float, PASN float, PASV float, PASX float, PASY float, PATG float, PATK float, PATL flo

In [42]:
isdm.create_isd_meta()


        CREATE TABLE IF NOT EXISTS air_meta 
        (id integer,
        name varchar(100),
        city varchar(100),
        state char(2),
        callsign char(4),
        usaf integer,
        wban integer,
        lat float,
        lng float);
        


In [8]:
r0 = sqldr.get_data("SELECT * from air_meta WHERE air_meta.state = 'ID' LIMIT 500;")

In [9]:
len(r0)

11

In [39]:
# Need to handle quotes for single/double quotes in strings.  
#sqldr.rollback()

In [44]:
# populate SQL table with metadata on ISD airports.
#isdm=populate_isd_meta()

In [14]:
#r0 = sqldr.get_data("SELECT * FROM information_schema.tables;")

In [10]:
# Get columns for a given table
#sqldr.get_columns('temperature')

In [12]:
from tzfpy import get_tz

In [13]:
get_tz

<function tzfpy.tzfpy.get_tz(lng, lat)>

In [11]:
isdm.load_data(Nst=2, Nmax=5)

['INSERT INTO temperature(ts,K79J) VALUES(1420074000000000000, 100),\n(1420077600000000000, 94),\n(1420081200000000000, 94),\n(1420084800000000000, 83),\n(1420088400000000000, 72)ON CONFLICT (ts) DO UPDATE SET K79J=EXCLUDED.K79J;',
 'INSERT INTO wind_dir(ts,K79J) VALUES(1420074000000000000, 80.0),\n(1420077600000000000, 80.0),\n(1420081200000000000, 0.0),\n(1420084800000000000, 0.0),\n(1420088400000000000, 0.0)ON CONFLICT (ts) DO UPDATE SET K79J=EXCLUDED.K79J;',
 'INSERT INTO wind_speed(ts,K79J) VALUES(1420074000000000000, 26),\n(1420077600000000000, 15),\n(1420081200000000000, 0),\n(1420084800000000000, 0),\n(1420088400000000000, 0)ON CONFLICT (ts) DO UPDATE SET K79J=EXCLUDED.K79J;',
 'INSERT INTO precip_1hr(ts,K79J) VALUES(1420074000000000000, nan),\n(1420077600000000000, nan),\n(1420081200000000000, nan),\n(1420084800000000000, nan),\n(1420088400000000000, nan)ON CONFLICT (ts) DO UPDATE SET K79J=EXCLUDED.K79J;',
 'INSERT INTO temperature(ts,K79J) VALUES(1451610000000000000, 144),\n(

In [10]:
# Load in EBA data
def wrap():
    import ipdb; ipdb.set_trace()
    isdm.load_data(Nst=2, Nmax=5)

wrap()
# iterate over files. 

> /tmp/ipykernel_676/3710667136.py(4)wrap()
      3     import ipdb; ipdb.set_trace()
----> 4     isdm.load_data(Nst=2, Nmax=5)
      5 

ipdb> s
--Call--
> /tf/us_elec/SQL/sqldriver.py(300)load_data()
    299 
--> 300     def load_data(self, Nst=-1, Nmax=-1):
    301         """Load data for each station by year and insert desired data into columns of relevant tables.

ipdb> n
> /tf/us_elec/SQL/sqldriver.py(305)load_data()
    304         """
--> 305         files = self.get_isd_filenames()[:Nst]
    306 

ipdb> 
> /tf/us_elec/SQL/sqldriver.py(307)load_data()
    306 
--> 307         out_sql = []
    308         for file, callsign, tzstr in files:

ipdb> n
> /tf/us_elec/SQL/sqldriver.py(308)load_data()
    307         out_sql = []
--> 308         for file, callsign, tzstr in files:
    309             df = load_isd_df(file, tzstr)

ipdb> n
> /tf/us_elec/SQL/sqldriver.py(309)load_data()
    308         for file, callsign, tzstr in files:
--> 309             df = load_isd_df(file, tzstr

ipdb> 
> /tf/us_elec/SQL/sqldriver.py(312)load_data()
    311                 data = self.get_df_data_cols(df, df_col)[:Nmax]
--> 312                 data_types = ("datetime", data_type)
    313                 cols = ["ts", callsign]

ipdb> 
> /tf/us_elec/SQL/sqldriver.py(313)load_data()
    312                 data_types = ("datetime", data_type)
--> 313                 cols = ["ts", callsign]
    314                 s_c = self.sqldr.upsert_data_column(sql_table, cols, data_types, data)

ipdb> 
> /tf/us_elec/SQL/sqldriver.py(314)load_data()
    313                 cols = ["ts", callsign]
--> 314                 s_c = self.sqldr.upsert_data_column(sql_table, cols, data_types, data)
    315                 out_sql.append(s_c)

ipdb> 
> /tf/us_elec/SQL/sqldriver.py(315)load_data()
    314                 s_c = self.sqldr.upsert_data_column(sql_table, cols, data_types, data)
--> 315                 out_sql.append(s_c)
    316         return out_sql

ipdb> 
> /tf/us_elec/SQL/sqldriver.py(

In [26]:
import os
from us_elec.SQL.sqldriver import ISDName

all_files = os.listdir('/tf/data/ISD')

wban_usaf_list = sqldr.get_data(
    f"SELECT USAF, WBAN, CALLSIGN FROM {ISDName.META}"
)


In [42]:
expect_fn = isdm.get_isd_filenames()
get_fn = lambda x: x[0].split('/')[-1]
found_fn = [get_fn(x) for x in expect_fn]

In [43]:
len(expect_fn)

7570

In [44]:
len(all_files)

7583

In [ ]:
TODO:
    - Fix reading in data to just use a simple reader without conversion to pandas.
    - Need to convert ISD timezone to UTC.  